In [1]:
import dotenv
dotenv.load_dotenv()

True

In [2]:
import yaml
from datasets import Dataset
from trl import KTOTrainer, KTOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig

[2024-06-15 15:38:29,345] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [3]:
from typing import Dict, Union, Any, Tuple
import torch
from torch import nn
import warnings
from contextlib import nullcontext
from transformers import PreTrainedModel
class MyKTOTrainer(KTOTrainer):
    def compute_loss(
        self,
        model: Union[PreTrainedModel, nn.Module],
        inputs: Dict[str, Union[torch.Tensor, Any]],
        return_outputs=False,
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, Dict[str, torch.Tensor]]]:
        if not self.use_dpo_data_collator:
            warnings.warn(
                "compute_loss is only implemented for DPODataCollatorWithPadding, and you passed a datacollator that is different than "
                "DPODataCollatorWithPadding - you might see unexpected behavior. Alternatively, you can implement your own prediction_step method if you are using a custom data collator"
            )
        compute_loss_context_manager = torch.cuda.amp.autocast if self._peft_has_been_casted_to_bf16 else nullcontext

        with compute_loss_context_manager():
            loss, metrics = self.get_batch_loss_metrics(model, inputs)

        # Make sure to move the loss to the device the original accumulating loss is at back in the `Trainer` class:
        loss = loss.to(self.args.device)
        # force log the metrics
        if self.accelerator.is_main_process:
            self.store_metrics(metrics, train_eval="train")

        loss.requires_grad_(True) # this is the only change from the original code
        if return_outputs:
            return (loss, metrics)
        return loss

In [4]:
model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'
checkpoint = yaml.safe_load(open('checkpoint_kto_dataset_python_cot.yaml'))
output_path = 'kto_output'
dataset = Dataset.from_dict(
    {
        'prompt': checkpoint['prompts'],
        'completion': checkpoint['completions'],
        'label': checkpoint['labels'],
    }
)

In [5]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype='bfloat16',
) 

model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

split = dataset.train_test_split(test_size=0.1)
train_dataset = split['train']
test_dataset = split['test']


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
trainer = MyKTOTrainer(
    model,
    None,
    args=KTOConfig(
        output_dir=output_path,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        bf16=True,
        gradient_accumulation_steps=8, 
        do_eval=True,
        logging_strategy='steps',
        save_strategy='steps',
        evaluation_strategy='steps',
        logging_steps=20,
        save_steps=20,
        eval_steps=20,
        num_train_epochs=10,
        load_best_model_at_end=True,
        warmup_steps=20,
        
        loss_type='bco'
    ),
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:464: UserWarning: When using DPODataCollatorWithPadding, you should set `max_length` in the KTOTrainer's init it will be set to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:474: UserWarning: When using DPODataCollatorWithPadding, you should set `max_prompt_length` in the KTOTrainer's init it will be set to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:504: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your KTOConfig we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Extracting KL train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Processing tokenized train dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

Processing tokenized train KL dataset:   0%|          | 0/633 [00:00<?, ? examples/s]

...

Processing tokenized eval dataset:   0%|          | 0/71 [00:00<?, ? examples/s]

Processing tokenized eval KL dataset:   0%|          | 0/71 [00:00<?, ? examples/s]

Filtering desirable examples:   0%|          | 0/633 [00:00<?, ? examples/s]

Filtering undesirable examples:   0%|          | 0/633 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/kto_trainer.py:660: UserWarning: 
                        You have different amounts of desirable/positive and undesirable/negative examples but the
                        weights on the desirable and undesirable losses don't seem to be in an ideal range. Based
                        on your data, we recommend EITHER desirable_weight in [1.33, 1.77]
                        or undesirable_weight in [0.57, 0.75] (but NOT BOTH).
                        See the documentation on how to optimally set these weights.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=No

In [7]:
trainer.train()

wandb: Currently logged in as: chengpong1127. Use `wandb login --relogin` to force relogin


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/chosen,Logps/chosen,Rewards/rejected,Logps/rejected,Rewards/margins,Kl
20,0.693100,0.693147,0.000000,-183.387812,0.000000,-250.126445,0.000000,0.000000
40,0.693100,0.693147,0.000000,-183.387812,0.000000,-250.126445,0.000000,0.000000
60,0.693100,0.693147,0.000000,-183.387812,0.000000,-250.126445,0.000000,0.000000
80,0.693100,0.693147,0.000000,-183.387812,0.000000,-250.126445,0.000000,0.000000
100,0.693100,0.693147,0.000000,-183.387812,0.000000,-250.126445,0.000000,0.000000


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-666db8f1-17b668f3602b906819eaf11c;3b676c8b-20a9-45e7-ba0b-a3a43ac539e8)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B-Instruct is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B-Instruct.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-666dbc5b-0cec671355c09e4

TrainOutput(global_step=110, training_loss=0.6931463241577148, metrics={'train_runtime': 4782.0291, 'train_samples_per_second': 1.51, 'train_steps_per_second': 0.023, 'total_flos': 0.0, 'train_loss': 0.6931463241577148, 'epoch': 9.67})